In [20]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import cv2
import numpy as np
import glob

#List tiff file
patchfile = glob.glob("*.tiff")

for impatch in patchfile:

    #Import image
    im = Image.open(impatch)

    #Set to array
    imarray = np.array(im)

    #Delete R B color
    gimarray = imarray.copy()
    gimarray[:, :, 0] = 0
    gimarray[:, :, 2] = 0

    #Fillter noise
    gimarray[gimarray < 30] = 0

    #Split G color array
    gareaimarray = gimarray[:, :, 1]
    cvgareaimarray = gareaimarray.copy() #Bufer for cv process

    #Calculate intentsive of green area
    gareaimarray = gareaimarray[gareaimarray > 0]
    print(impatch)
    print(gareaimarray.mean())

    #combine 2 image
    imarray = np.concatenate((imarray, np.full((1024, 2, 3), 255, dtype="uint8"), gimarray), axis=1)
    
    #Find number of circle
    circles = cv2.HoughCircles(cvgareaimarray,
                           cv2.HOUGH_GRADIENT,
                           minDist=35,
                           dp=1,
                           param1=150,
                           param2=12,
                           minRadius=12,
                           maxRadius=30)

    i = 0
    for (x, y, r) in circles[0, :, :]:
        i = i + 1
        cv2.circle(imarray, (x, y), r, (0, 255, 0), 4)
        cv2.putText(imarray,str(i), (x,y), cv2.FONT_HERSHEY_PLAIN, fontScale=1.2,
                    color=(255, 255, 255), thickness=2)
    
    #Draw text in photo
    Finalim = Image.fromarray(imarray)
    draw = ImageDraw.Draw(Finalim)
    font = ImageFont.truetype('Montserrat-Black.ttf', 20)
    text = "Mean avg. : " + str(gareaimarray.mean()/3) + "\nTotal IntDen : " + str(gareaimarray.sum()/3)
    draw.text((1031, 5), text, (255,255,255), font=font)

    #Save image
    Finalim.save("Split" + impatch[:-5] + ".jpg")


Tol4-1 high.tiff
65.41690352335496
TolC251-1.tiff
52.72097145896118
